#1-Issues : Collect , Clean and preprocess the dataset

In [10]:
##import dependencies
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
## Let's import api json file
from google.colab import files
files.upload()

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
## Download the data from kaggle
!kaggle datasets download -d harits/anime-database-2022

  0% 0.00/5.31M [00:00<?, ?B/s]
100% 5.31M/5.31M [00:00<00:00, 189MB/s]


In [5]:
##Unzip the downloaded data
!unzip -qq anime-database-2022.zip

In [8]:
##open the anime.csv file as pandas dataframe 
anime_data = pd.read_csv('/content/Anime.csv')

In [9]:
## see first lines of our data
anime_data.head()

,ID,Title,Synonyms,Japanese,English,Synopsis,Type,Episodes,Status,Start_Aired,...,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
0,16498,Shingeki no Kyojin,"AoT, SnK",進撃の巨人,Attack on Titan,"Centuries ago, mankind was slaughtered to near...",TV,25.0,Finished Airing,"Apr 7, 2013",...,"Gore, Military, Survival",Shounen,24.0,R - 17+ (violence & profanity),8.531,519803.0,1002.0,1,3524109,155695
1,1535,Death Note,DN,デスノート,Death Note,"Brutal murders, petty thefts, and senseless vi...",TV,37.0,Finished Airing,"Oct 4, 2006",...,Psychological,Shounen,23.0,R - 17+ (violence & profanity),8.621,485487.0,732.0,2,3504535,159701
2,5114,Fullmetal Alchemist: Brotherhood,"Hagane no Renkinjutsushi Fullmetal Alchemist, ...",鋼の錬金術師 FULLMETAL ALCHEMIST,Fullmetal Alchemist Brotherhood,After a horrific alchemy experiment goes wrong...,TV,64.0,Finished Airing,"Apr 5, 2009",...,Military,Shounen,24.0,R - 17+ (violence & profanity),9.131,900398.0,12.0,3,2978455,207772
3,30276,One Punch Man,"One Punch-Man, One-Punch Man, OPM",ワンパンマン,One Punch Man,The seemingly unimpressive Saitama has a rathe...,TV,12.0,Finished Airing,"Oct 5, 2015",...,"Parody, Super Power",Seinen,24.0,R - 17+ (violence & profanity),8.511,19066.0,1112.0,4,2879907,59651
4,11757,Sword Art Online,"S.A.O, SAO",ソードアート・オンライン,Sword Art Online,Ever since the release of the innovative Nerve...,TV,25.0,Finished Airing,"Jul 8, 2012",...,"Love Polygon, Video Game",Unknown,23.0,PG-13 - Teens 13 or older,7.201,990254.0,29562.0,5,2813565,64997


In [ ]:
## find rows where both genres and Themes are unknown
#Themes and Genres are ones the most informative columns for our system
#so if both are empty for a certain anime, we won't get very interesting recommandation for them
#We will deal with other Features Later

In [68]:
tmp_df = anime_data[(anime_data['Genres'] == 'Unknown') & (anime_data['Themes'] == 'Unknown'  ) ]
tmp_df

,ID,Title,Synonyms,Japanese,English,Synopsis,Type,Episodes,Status,Start_Aired,...,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
4590,9911,Houkago no Pleiades,"Afterschool Pleiades, Hokago no Pleiades",放課後のプレアデス,Wish Upon the Pleiades,Subaru is a young girl who likes to see the st...,ONA,4.0,Finished Airing,"Feb 1, 2011",...,Unknown,Unknown,5.0,G - All Ages,6.131,8701.0,84102.0,4595,17318,13
4703,21421,Mahou Shoujo Taisen,Unknown,魔法少女大戦,Magica Wars,"""Mahou Shoujo Taisen"" is a project created fro...",TV,26.0,Finished Airing,"Apr 8, 2014",...,Unknown,Unknown,4.0,PG-13 - Teens 13 or older,5.751,5720.0,101362.0,4708,16302,5
4817,51092,Yurei Deco,You0 DECO,ユーレイデコ,Yurei Deco,"In the cyber city Tom Sawyer, society revolves...",TV,12.0,Finished Airing,"Jul 3, 2022",...,Unknown,Unknown,23.0,PG-13 - Teens 13 or older,6.141,3833.0,83042.0,4822,15293,34
5018,6399,Higashi no Eden: Falling Down,Oasis Falling Down,フォーリング・ダウン,Unknown,"Music video by Oasis, animated by Production I...",Music,1.0,Finished Airing,"May 27, 2009",...,Unknown,Unknown,4.0,G - All Ages,7.361,7623.0,21742.0,5023,13700,15
5136,21771,Romantica Clock,"Ribbon x Oha Suta Specials, Ribon x Oha Suta S...",ロマンチカ クロック,Unknown,Three episodes aired on TV as part of the Ribo...,Special,3.0,Finished Airing,"Feb 4, 2014",...,Unknown,Shoujo,1.0,G - All Ages,5.701,6518.0,103482.0,5141,13004,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21407,46540,Konglong Baobao Man Youji: Wenming Baodian,Unknown,恐龙宝宝漫游记之文明宝典,Unknown,Unknown,TV,72.0,Finished Airing,2014,...,Unknown,Kids,7.0,PG - Children,NaN,NaN,193112.0,21439,21,0
21408,46541,Konglong Baobao Man Youji: Wo Ai Wojia,Unknown,恐龙宝宝漫游记之我爱我家,Unknown,Unknown,TV,144.0,Finished Airing,Unknown,...,Unknown,Kids,7.0,PG - Children,NaN,NaN,193122.0,21440,21,0
21413,45907,Mao Er Shan de Guizi Bing,Unknown,帽儿山的鬼子兵,Unknown,Unknown,TV,36.0,Finished Airing,"Feb 4, 2006",...,Unknown,Kids,15.0,PG - Children,NaN,NaN,132852.0,21444,21,0
21442,46761,Xiaoqiang Yijia,Unknown,小强一家,Unknown,Unknown,TV,6.0,Finished Airing,2013,...,Unknown,Kids,20.0,PG - Children,NaN,NaN,161812.0,21474,15,0


In [75]:
#Now let's drop rows where both genres and themes are 'Unknown

In [71]:
len(anime_data), len(tmp_df)

(21460, 1175)

In [76]:
cleaned_anime_data = pd.concat([anime_data, tmp_df, tmp_df]).drop_duplicates(keep=False)
len(cleaned_anime_data)

20285

In [79]:
len(cleaned_anime_data[(cleaned_anime_data['Themes'] == 'Unknown') & (cleaned_anime_data['Genres'] == 'Unknown' ) ])

0

In [80]:
## Unknown rows have been removed
## Lets now select the columns we will use as our features

In [81]:
cleaned_anime_data.columns

Index(['ID', 'Title', 'Synonyms', 'Japanese', 'English', 'Synopsis', 'Type',
       'Episodes', 'Status', 'Start_Aired', 'End_Aired', 'Premiered',
       'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres',
       'Themes', 'Demographics', 'Duration_Minutes', 'Rating', 'Score',
       'Scored_Users', 'Ranked', 'Popularity', 'Members', 'Favorites'],
      dtype='object')

In [91]:
features_list = ['Producers', 'Licensors', 'Studios', 'Genres', 'Themes']

In [92]:
##let's do a bit of cleaning our features
##Let's find how many Unknown rows we have in each feature

In [149]:
df = cleaned_anime_data
for feature in features_list:
  len_unknown = len(df[df[feature] == 'Unknown' ])
  print(f'{feature} number of Unknown Rows : {len_unknown} ')

Producers number of Unknown Rows : 9566 
Licensors number of Unknown Rows : 15713 
Studios number of Unknown Rows : 7308 
Genres number of Unknown Rows : 2524 
Themes number of Unknown Rows : 8473 


In [94]:
#We are going to replace unknow value with empty space 
#to avoid corrupting our vectorisation

In [150]:
for feature in features_list:
  df[feature] = df[feature].replace(['Unknown'], '')  

for feature in features_list:
  len_unknown = len(df[df[feature] == 'Unknown' ])
  print(f'{feature} number of Unknown Rows : {len_unknown} ')


Producers number of Unknown Rows : 0 
Licensors number of Unknown Rows : 0 
Studios number of Unknown Rows : 0 
Genres number of Unknown Rows : 0 
Themes number of Unknown Rows : 0 


In [151]:
len(df)

20285

#2-Issues : Build a scratch recommander as baseline

In [154]:
## Now let's combine our features in a single column for vectortizing 
combined_anime_data = df['Producers'] + ' ' + df['Licensors'] + ' ' + df['Studios'] + ' ' + df['Genres'] + ' ' + df['Themes']

In [155]:
combined_anime_data

0        Production I.G, Dentsu, Mainichi Broadcasting ...
1        VAP, Konami, Ashi Productions, Nippon Televisi...
2        Aniplex, Square Enix, Mainichi Broadcasting Sy...
3        TV Tokyo, Bandai Visual, Lantis, Asatsu DK, Ba...
4        Aniplex, Genco, DAX Production, ASCII Media Wo...
                               ...                        
21455                                  Adventure, Fantasy 
21456                                  Adventure, Fantasy 
21457                          Adventure, Comedy, Fantasy 
21458                                  Adventure, Fantasy 
21459                                 Adventure Historical
Length: 20285, dtype: object

In [156]:
## Now let's vectorize our data with hte tfidfVectorizer we import eaelier
Vectorizer = TfidfVectorizer()
vect_anime_data = Vectorizer.fit_transform(combined_anime_data)
print(vect_anime_data)

  (0, 1888)	0.2559680819696681
  (0, 1276)	0.1650638664121932
  (0, 779)	0.22585546181069135
  (0, 537)	0.1109005963500285
  (0, 32)	0.09412681940332851
  (0, 1862)	0.11931056109752539
  (0, 2126)	0.2454807329481692
  (0, 706)	0.14156397287288017
  (0, 607)	0.24398893758164675
  (0, 252)	0.2748896990702373
  (0, 1190)	0.3530546233024179
  (0, 1083)	0.19451319299190245
  (0, 291)	0.3701241103657919
  (0, 1550)	0.3701241103657919
  (0, 1898)	0.19871615576471535
  (0, 265)	0.18486397318659664
  (0, 1208)	0.20033008904928323
  (0, 491)	0.18689413850459724
  (0, 1574)	0.14227662441204092
  (1, 1582)	0.24184338771899241
  (1, 1890)	0.2795765398278845
  (1, 1887)	0.17505468736126223
  (1, 1191)	0.2444854521028495
  (1, 1251)	0.17675023351026387
  (1, 2087)	0.27801710809716107
  :	:
  (20272, 640)	0.4514663712846701
  (20273, 118)	0.8922881348527798
  (20273, 640)	0.4514663712846701
  (20274, 865)	1.0
  (20275, 118)	0.8922881348527798
  (20275, 640)	0.4514663712846701
  (20276, 118)	0.89228813

In [157]:
# find cosine_similarity confidence for each row of data
#This will output for each row in our dataset the similarity score in regard to all others rows
similarity = cosine_similarity(vect_anime_data)

In [160]:
#as we aregoing to take a user input and find the highest cosine_similarity value we have to expect that user will make typos in anime name
#We will use the difflib package to match the closest anime title to the user input

list_of_titles = cleaned_anime_data.Title.tolist()
print(list_of_titles)

['Shingeki no Kyojin', 'Death Note', 'Fullmetal Alchemist: Brotherhood', 'One Punch Man', 'Sword Art Online', 'Boku no Hero Academia', 'Naruto', 'Tokyo Ghoul', 'Kimetsu no Yaiba', 'Hunter x Hunter (2011)', 'Kimi no Na wa.', 'Shingeki no Kyojin Season 2', 'Steins;Gate', 'Boku no Hero Academia 2nd Season', 'No Game No Life', 'Naruto: Shippuuden', 'Code Geass: Hangyaku no Lelouch', 'Shingeki no Kyojin Season 3', 'Koe no Katachi', 'Toradora!', 'Shigatsu wa Kimi no Uso', 'Jujutsu Kaisen', 'Noragami', 'Re:Zero kara Hajimeru Isekai Seikatsu', 'One Piece', 'Boku no Hero Academia 3rd Season', 'Angel Beats!', 'Shingeki no Kyojin Season 3 Part 2', 'Akame ga Kill!', 'Boku dake ga Inai Machi', 'Nanatsu no Taizai', 'Mirai Nikki (TV)', 'Sword Art Online II', 'Mob Psycho 100', 'Ansatsu Kyoushitsu', 'Ao no Exorcist', 'Haikyuu!!', 'Kono Subarashii Sekai ni Shukufuku wo!', 'Yakusoku no Neverland', 'Bleach', 'Kiseijuu: Sei no Kakuritsu', 'Shingeki no Kyojin: The Final Season', 'Cowboy Bebop', 'Sen to Chih

In [161]:
##get user input and find corresponding anime with difflib
user_input = input('Enter your anime :')

Enter your anime :Tokyo Goul


In [162]:
matchs = difflib.get_close_matches(user_input, list_of_titles)
print(matchs)

['Tokyo Ghoul', 'Tokyo Ghoul:re', 'Tokyo Ghoul √A']


In [163]:
## as expected we have a list of the corresponding titles
# remark that i voluntarly made a typo in the anime name but we still get a good result

In [165]:
closest_match = matchs[0]
print(closest_match)

Tokyo Ghoul


In [177]:
#We're goign to use the index of the anime to match it later with his similarity score 
idx = cleaned_anime_data[cleaned_anime_data.Title == closest_match]['index'].values[0]

In [178]:
idx

7

In [180]:
##get a similarity value for the anime with his index in regard to every other animes in our dataset

similarity_score = list(enumerate(similarity[idx]))
print(similarity_score)

[(0, 0.11442529395741718), (1, 0.12495354225767123), (2, 0.0691666627896056), (3, 0.05995707845988733), (4, 0.029793661548429717), (5, 0.13658544626791985), (6, 0.2367655742180169), (7, 1.0000000000000002), (8, 0.13291587164482557), (9, 0.12720712459472158), (10, 0.023956075057572093), (11, 0.11766942924744792), (12, 0.08979732795198203), (13, 0.16084779659083848), (14, 0.019649802024648333), (15, 0.17088141718000835), (16, 0.0987132579126756), (17, 0.11766942924744792), (18, 0.0), (19, 0.0), (20, 0.022712511786662812), (21, 0.09992486445923053), (22, 0.1089215194363955), (23, 0.0915481700751753), (24, 0.20037265224874085), (25, 0.16084779659083848), (26, 0.0), (27, 0.11766942924744792), (28, 0.1469122988559578), (29, 0.04260832331962919), (30, 0.09652861044656036), (31, 0.12189081949209725), (32, 0.03113752680106841), (33, 0.05236329952215303), (34, 0.11454892886462817), (35, 0.09190928671722312), (36, 0.08504670568601111), (37, 0.013015893212533873), (38, 0.10282282979514457), (39, 0

In [181]:
#remark that we have a similarity score with regard to all other anime - mangas
#We will only recommand the most similar anime to user

len(similarity_score)

20285

In [182]:
sorted_similarity_score = sorted(similarity_score, key = lambda x : x[1], reverse = True)

In [183]:
sorted_similarity_score

[(7, 1.0000000000000002),
 (44, 0.8690863935275862),
 (200, 0.8230579619386555),
 (116, 0.7842492530023923),
 (1035, 0.6248648469418094),
 (876, 0.5999931327435131),
 (17922, 0.4349793213149901),
 (7362, 0.42155949045260727),
 (11296, 0.37390086446901705),
 (6762, 0.3597644076581428),
 (4607, 0.3576939590564542),
 (19261, 0.3549949750768521),
 (19327, 0.3549949750768521),
 (1268, 0.3543801083146348),
 (1488, 0.3501667482664947),
 (1253, 0.3436513601807758),
 (5562, 0.3426656289034784),
 (2732, 0.33815509968083574),
 (2739, 0.33815509968083574),
 (1120, 0.33380540211317655),
 (5934, 0.32761213810410456),
 (2498, 0.3251968724386771),
 (1884, 0.3228867891415227),
 (138, 0.318271827673436),
 (2020, 0.3156991887775307),
 (900, 0.3155776061755318),
 (1938, 0.3141873442866385),
 (11191, 0.3116572958049022),
 (5243, 0.3105220336871481),
 (15326, 0.31011904142577323),
 (2124, 0.3094227837559025),
 (9314, 0.30907175024330846),
 (3658, 0.3076931689383486),
 (18969, 0.3056104355103837),
 (4640, 0.

In [186]:
#Now let's generate recommendations for the user input
i = 1
for item in sorted_similarity_score:
	idx = item[0]
	title_from_idx = cleaned_anime_data[cleaned_anime_data.index == idx]['Title'].values[0]
	if (i<30):
		print(title_from_idx)
		i += 1

Tokyo Ghoul
Tokyo Ghoul √A
Tokyo Ghoul:re 2nd Season
Tokyo Ghoul:re
Tokyo Ghoul: "Pinto"
Tokyo Ghoul: "Jack"
Ryoku Tama Shinshi
Precure All Stars Movie New Stage: Mirai no Tomodachi
Toilet no Hanako-san
Saint Beast: Kouin Jojishi Tenshi Tan
Senran Kagura Specials
Atama
Papa ga Tonda Asa
Bleach: Sennen Kessen-hen
Senran Kagura
Divine Gate
Yume-iro Pâtissière: Mune Kyun Tropical Island!
Naruto: The Cross Roads
Naruto x UT
Gunslinger Girl
Yuu☆Yuu☆Hakusho: Mu Mu Hakusho - Nightmare Hakusho
Shin Tennis no Ouji-sama
Higurashi no Naku Koro ni Sotsu
Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku
Oretachi ni Tsubasa wa Nai: Under the Innocent Sky.
Bleach Movie 2: The DiamondDust Rebellion - Mou Hitotsu no Hyourinmaru
Hellsing: The Dawn
Yumemakura Baku Twilight Gekijou
Yuu☆Yuu☆Hakusho: Eizou Hakusho - Ankoku Bujutsukai no Shou


In [ ]:
###Lets test it out 

In [192]:
##get user input and find corresponding anime with difflib
user_input = input('Enter your anime :')
matchs = difflib.get_close_matches(user_input, list_of_titles)
closest_match = matchs[0]

print(f'closest_match : {closest_match}')
idx = cleaned_anime_data[cleaned_anime_data.Title == closest_match]['index'].values[0]
##get a similarity value for the anime with his index in regard to every other animes in our dataset

similarity_score = list(enumerate(similarity[idx]))
sorted_similarity_score = sorted(similarity_score, key = lambda x : x[1], reverse = True)
i = 1
for item in sorted_similarity_score:
	idx = item[0]
	title_from_idx = cleaned_anime_data[cleaned_anime_data.index == idx]['Title'].values[0]
	if (i<30):
		print(title_from_idx)
		i += 1

Enter your anime :kimetsu no yaiba
closest_match : Kimetsu no Yaiba
Kimetsu no Yaiba
Kimetsu no Yaiba Movie: Mugen Ressha-hen
Kimetsu no Yaiba: Yuukaku-hen
Kara no Kyoukai: Manner Movies
Fate/Zero: Onegai! Einzbern Soudanshitsu
Kimetsu no Yaiba: Mugen Ressha-hen
Kara no Kyoukai Movie: Mirai Fukuin - Manner Movie
Gyo
Fate/stay night: Unlimited Blade Works 2nd Season - Sunny Day
Kimetsu no Yaiba: Katanakaji no Sato-hen
Fate/Zero Remix
Fate/stay night: Unlimited Blade Works
Fate/stay night: Unlimited Blade Works 2nd Season
Fate/stay night: Unlimited Blade Works Prologue
Kara no Kyoukai Movie 8: Shuushou
Mashle
Ao no Exorcist Movie
Fate/stay night Movie: Heaven's Feel - I. Presage Flower
Fate/stay night Movie: Heaven's Feel - II. Lost Butterfly
Fate/stay night Movie: Heaven's Feel - III. Spring Song
Ao no Exorcist: Kuro no Iede
Kara no Kyoukai Movie: Mirai Fukuin - Extra Chorus
Minori Scramble!
Ao no Exorcist: Ura Ex
Kara no Kyoukai Remix: Gate of Seventh Heaven
Emiya-san Chi no Kyou no Go

In [ ]:
## That is not very bad for a first baseline, we can output some useful recommdations 
#A lot more data will be very useful as Genre , Themes , Studios for all the rows